## Setting up the notebook

In [ ]:
# installing packages
!pip install pip==24.0
!pip install numpy==1.23.5
!pip install tensorboardX
!pip install subword-nmt
!pip install sentencepiece

  Using cached tensorboardX-2.6.2.2-py2.py3-none-any.whl.metadata (5.8 kB)
Using cached tensorboardX-2.6.2.2-py2.py3-none-any.whl (101 kB)


In [ ]:
# importing packages
import numpy
import os
import tensorboardX
import sentencepiece as spm

In [ ]:
# mounting google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
source_code = 'eng'
target_code = 'tso'

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}')

# installing fairseq
#!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
!pip install --editable ./

/content/drive/MyDrive/Research/eng-to-tso/fairseq
Obtaining file:///content/drive/MyDrive/Research/eng-to-tso/fairseq
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.3/278.3 kB 6.0 MB/s eta 0:00:00
  Building editable for fairseq (pyproject.toml) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-0.editable-cp310-cp310-linux_x86_64.whl size=9581 sha256=98482531860113099fde0fbf0da1f1ae6574f6afff29e0788afb9e2b22438c

## Evaluate Model on Raw Test and Global Test Sets

In [ ]:
import sacrebleu

In [ ]:
before_punc = '!,.:;?)'
def remove_whitespace_before(segment, punc):
  """
  This function looks for punctuation in a segment and removes the whitespace
  before the punctuation mark.

  Args:
    segment (str): segment to be processed.
    punc (str): string of punctuation marks.

  Returns:
    The segment with whitespace removed before punctuation marks in punc.
  """
  punc_positions = []
  for i, symbol in enumerate(segment):
    if symbol in punc:
      punc_positions.append(i)

  punc_positions.reverse()
  for pos in punc_positions:
    if segment[pos-1] == " ":
      segment = segment[0:pos-1] + segment[pos:]

  return segment

### BPE

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/bpe')

In [ ]:
!mkdir -p trained_model
!cp data-bin/dict.eng.txt trained_model/dict.eng.txt
!cp data-bin/dict.tso.txt trained_model/dict.tso.txt
!cp data/bpe.codes.4000 trained_model/bpe.codes.4000
!cp checkpoints-bpe/checkpoint_best.pt trained_model/model.pt

In [ ]:
!fairseq-interactive \
--path trained_model/model.pt \
--source-lang eng \
--target-lang tso \
--bpe 'subword_nmt' \
--bpe-codes trained_model/bpe.codes.4000 \
--beam 5 \
--lenpen 1 \
--seed 2024 \
--max-len-a 0 \
--max-len-b 100 \
--quiet \
--remove-bpe \
--buffer-size 100 \
--input /content/drive/MyDrive/Research/eng-to-tso/cleaned-data/clean_gtest.eng \
trained_model/ | grep -P "D-[0-9]+" | cut -f3 > trained_model/translations_gtest

2024-11-06 10:56:33.035930: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 10:56:33.061054: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 10:56:33.068532: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 10:56:33.086453: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 10:56:34.528550: W tensorflow/compiler/tf2

In [ ]:
!fairseq-interactive \
--path trained_model/model.pt \
--source-lang eng \
--target-lang tso \
--bpe 'subword_nmt' \
--bpe-codes trained_model/bpe.codes.4000 \
--beam 5 \
--lenpen 1 \
--seed 2024 \
--max-len-a 0 \
--max-len-b 100 \
--quiet \
--remove-bpe \
--buffer-size 100 \
--input /content/drive/MyDrive/Research/eng-to-tso/cleaned-data/test.eng \
trained_model/ | grep -P "D-[0-9]+" | cut -f3 > trained_model/translations_test

2024-11-06 10:59:55.160062: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 10:59:55.199115: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 10:59:55.210870: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 10:59:55.237224: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 10:59:56.929944: W tensorflow/compiler/tf2

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/cleaned-data')

In [ ]:
translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/bpe/trained_model/translations_gtest'

In [ ]:
# post-processing predictions
with open(translations_path, 'r', encoding='utf-8') as f:
  segments = f.read().splitlines()

for i in range(len(segments)):
  segments[i] = remove_whitespace_before(segments[i], before_punc)

post_translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/bpe/trained_model/post_process_translations_gtest'
with open(post_translations_path, 'w') as f:
  for segment in segments:
    f.write(segment + '\n')

In [ ]:
translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/bpe/trained_model/translations_test'

In [ ]:
# post-processing predictions
with open(translations_path, 'r', encoding='utf-8') as f:
  segments = f.read().splitlines()

for i in range(len(segments)):
  segments[i] = remove_whitespace_before(segments[i], before_punc)

post_translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/bpe/trained_model/post_process_translations_test'
with open(post_translations_path, 'w') as f:
  for segment in segments:
    f.write(segment + '\n')

### ULM

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/ulm')

In [ ]:
!mkdir -p trained_model
!cp data-bin/dict.eng.txt trained_model/dict.eng.txt
!cp data-bin/dict.tso.txt trained_model/dict.tso.txt
!cp data/joint.model trained_model/joint.model
!cp checkpoints-ulm/checkpoint_best.pt trained_model/model.pt

In [ ]:
!fairseq-interactive \
--path trained_model/model.pt \
--source-lang eng \
--target-lang tso \
--bpe 'sentencepiece' \
--sentencepiece-model trained_model/joint.model \
--beam 5 \
--lenpen 1 \
--seed 2024 \
--max-len-a 0 \
--max-len-b 100 \
--quiet \
--remove-bpe sentencepiece \
--buffer-size 100 \
--input /content/drive/MyDrive/Research/eng-to-tso/cleaned-data/clean_gtest.eng \
trained_model/ | grep -P "H-[0-9]+" | cut -f3 > trained_model/translations_gtest

2024-11-06 11:14:59.548822: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 11:14:59.585674: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 11:14:59.596221: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 11:14:59.622453: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 11:15:01.300566: W tensorflow/compiler/tf2

In [ ]:
!fairseq-interactive \
--path trained_model/model.pt \
--source-lang eng \
--target-lang tso \
--bpe 'sentencepiece' \
--sentencepiece-model trained_model/joint.model \
--beam 5 \
--lenpen 1 \
--seed 2024 \
--max-len-a 0 \
--max-len-b 100 \
--quiet \
--remove-bpe sentencepiece \
--buffer-size 100 \
--input /content/drive/MyDrive/Research/eng-to-tso/cleaned-data/test.eng \
trained_model/ | grep -P "H-[0-9]+" | cut -f3 > trained_model/translations_test

2024-11-06 11:18:37.822432: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 11:18:37.848920: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 11:18:37.856837: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 11:18:37.876961: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 11:18:39.307520: W tensorflow/compiler/tf2

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/cleaned-data')

In [ ]:
translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/ulm/trained_model/translations_gtest'

In [ ]:
# post-processing predictions
with open(translations_path, 'r', encoding='utf-8') as f:
  segments = f.read().splitlines()

for i in range(len(segments)):
  segments[i] = remove_whitespace_before(segments[i], before_punc)

post_translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/ulm/trained_model/post_process_translations_gtest'
with open(post_translations_path, 'w') as f:
  for segment in segments:
    f.write(segment + '\n')

In [ ]:
translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/ulm/trained_model/translations_test'

In [ ]:
# post-processing predictions
with open(translations_path, 'r', encoding='utf-8') as f:
  segments = f.read().splitlines()

for i in range(len(segments)):
  segments[i] = remove_whitespace_before(segments[i], before_punc)

post_translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/ulm/trained_model/post_process_translations_test'
with open(post_translations_path, 'w') as f:
  for segment in segments:
    f.write(segment + '\n')

### BPE-Dropout

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/bpeDROP')

In [ ]:
!mkdir -p trained_model
!cp data-bin-25/dict.eng.txt trained_model/dict.eng.txt
!cp data-bin-25/dict.tso.txt trained_model/dict.tso.txt
!cp /content/drive/MyDrive/Research/eng-to-tso/bpe/data/bpe.codes.4000 trained_model/bpe.codes.4000
!cp checkpoints-bpeDROP/checkpoint_best.pt trained_model/model.pt

In [ ]:
!fairseq-interactive \
--path trained_model/model.pt \
--source-lang eng \
--target-lang tso \
--bpe 'subword_nmt' \
--bpe-codes trained_model/bpe.codes.4000 \
--beam 5 \
--lenpen 1 \
--seed 2024 \
--max-len-a 0 \
--max-len-b 100 \
--quiet \
--remove-bpe \
--buffer-size 100 \
--input /content/drive/MyDrive/Research/eng-to-tso/cleaned-data/clean_gtest.eng \
trained_model/ | grep -P "D-[0-9]+" | cut -f3 > trained_model/translations_gtest

2024-11-06 11:27:12.736314: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 11:27:12.834626: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 11:27:12.852981: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 11:27:12.894281: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 11:27:16.884456: W tensorflow/compiler/tf2

In [ ]:
!fairseq-interactive \
--path trained_model/model.pt \
--source-lang eng \
--target-lang tso \
--bpe 'subword_nmt' \
--bpe-codes trained_model/bpe.codes.4000 \
--beam 5 \
--lenpen 1 \
--seed 2024 \
--max-len-a 0 \
--max-len-b 100 \
--quiet \
--remove-bpe \
--buffer-size 100 \
--input /content/drive/MyDrive/Research/eng-to-tso/cleaned-data/test.eng \
trained_model/ | grep -P "D-[0-9]+" | cut -f3 > trained_model/translations_test

2024-11-06 11:30:49.892408: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 11:30:49.919675: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 11:30:49.926953: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 11:30:49.945240: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 11:30:51.434453: W tensorflow/compiler/tf2

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/cleaned-data')

In [ ]:
translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/bpeDROP/trained_model/translations_gtest'

In [ ]:
# post-processing predictions
with open(translations_path, 'r', encoding='utf-8') as f:
  segments = f.read().splitlines()

for i in range(len(segments)):
  segments[i] = remove_whitespace_before(segments[i], before_punc)

post_translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/bpeDROP/trained_model/post_process_translations_gtest'
with open(post_translations_path, 'w') as f:
  for segment in segments:
    f.write(segment + '\n')

In [ ]:
translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/bpeDROP/trained_model/translations_test'

In [ ]:
# post-processing predictions
with open(translations_path, 'r', encoding='utf-8') as f:
  segments = f.read().splitlines()

for i in range(len(segments)):
  segments[i] = remove_whitespace_before(segments[i], before_punc)

post_translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/bpeDROP/trained_model/post_process_translations_test'
with open(post_translations_path, 'w') as f:
  for segment in segments:
    f.write(segment + '\n')

### ULM with Subword Regularization

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/ulmSR')

In [ ]:
!mkdir -p trained_model
!cp data-bin-25/dict.eng.txt trained_model/dict.eng.txt
!cp data-bin-25/dict.tso.txt trained_model/dict.tso.txt
!cp /content/drive/MyDrive/Research/eng-to-tso/ulm/data/joint.model trained_model/joint.model
!cp checkpoints-ulmSR/checkpoint_best.pt trained_model/model.pt

In [ ]:
!fairseq-interactive \
--path trained_model/model.pt \
--source-lang eng \
--target-lang tso \
--bpe 'sentencepiece' \
--sentencepiece-model trained_model/joint.model \
--beam 5 \
--lenpen 1 \
--seed 2024 \
--max-len-a 0 \
--max-len-b 100 \
--quiet \
--remove-bpe sentencepiece \
--buffer-size 100 \
--input /content/drive/MyDrive/Research/eng-to-tso/cleaned-data/clean_gtest.eng \
trained_model/ | grep -P "H-[0-9]+" | cut -f3 > trained_model/translations_gtest

2024-11-06 11:40:07.791521: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 11:40:07.816556: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 11:40:07.823744: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 11:40:07.841801: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 11:40:09.381211: W tensorflow/compiler/tf2

In [ ]:
!fairseq-interactive \
--path trained_model/model.pt \
--source-lang eng \
--target-lang tso \
--bpe 'sentencepiece' \
--sentencepiece-model trained_model/joint.model \
--beam 5 \
--lenpen 1 \
--seed 2024 \
--max-len-a 0 \
--max-len-b 100 \
--quiet \
--remove-bpe sentencepiece \
--buffer-size 100 \
--input /content/drive/MyDrive/Research/eng-to-tso/cleaned-data/test.eng \
trained_model/ | grep -P "H-[0-9]+" | cut -f3 > trained_model/translations_test

2024-11-06 11:44:07.283263: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 11:44:07.337922: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 11:44:07.354619: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 11:44:07.392532: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 11:44:09.633566: W tensorflow/compiler/tf2

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/cleaned-data')

In [ ]:
translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/ulmSR/trained_model/translations_gtest'

In [ ]:
# post-processing predictions
with open(translations_path, 'r', encoding='utf-8') as f:
  segments = f.read().splitlines()

for i in range(len(segments)):
  segments[i] = remove_whitespace_before(segments[i], before_punc)

post_translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/ulmSR/trained_model/post_process_translations_gtest'
with open(post_translations_path, 'w') as f:
  for segment in segments:
    f.write(segment + '\n')

In [ ]:
translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/ulmSR/trained_model/translations_test'

In [ ]:
# post-processing predictions
with open(translations_path, 'r', encoding='utf-8') as f:
  segments = f.read().splitlines()

for i in range(len(segments)):
  segments[i] = remove_whitespace_before(segments[i], before_punc)

post_translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/ulmSR/trained_model/post_process_translations_test'
with open(post_translations_path, 'w') as f:
  for segment in segments:
    f.write(segment + '\n')

### Task-specific Tokenization

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/target-tok')

In [ ]:
!mkdir -p trained_model
!cp data-bin-nmt/dict.eng.txt trained_model/dict.eng.txt
!cp data-bin-nmt/dict.tso.txt trained_model/dict.tso.txt
!cp /content/drive/MyDrive/Research/eng-to-tso/ulm/data/joint.model trained_model/joint.model
!cp checkpoints-task-nmt/checkpoint_best.pt trained_model/model.pt

In [ ]:
!fairseq-interactive \
--path trained_model/model.pt \
--source-lang eng \
--target-lang tso \
--bpe 'sentencepiece' \
--sentencepiece-model trained_model/joint.model \
--beam 5 \
--lenpen 1 \
--seed 2024 \
--max-len-a 0 \
--max-len-b 100 \
--quiet \
--remove-bpe sentencepiece \
--buffer-size 100 \
--input /content/drive/MyDrive/Research/eng-to-tso/cleaned-data/clean_gtest.eng \
trained_model/ | grep -P "H-[0-9]+" | cut -f3 > trained_model/translations_gtest

2024-11-06 11:57:37.132367: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 11:57:37.158400: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 11:57:37.165806: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 11:57:37.186094: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 11:57:38.641172: W tensorflow/compiler/tf2

In [ ]:
!fairseq-interactive \
--path trained_model/model.pt \
--source-lang eng \
--target-lang tso \
--bpe 'sentencepiece' \
--sentencepiece-model trained_model/joint.model \
--beam 5 \
--lenpen 1 \
--seed 2024 \
--max-len-a 0 \
--max-len-b 100 \
--quiet \
--remove-bpe sentencepiece \
--buffer-size 100 \
--input /content/drive/MyDrive/Research/eng-to-tso/cleaned-data/test.eng \
trained_model/ | grep -P "H-[0-9]+" | cut -f3 > trained_model/translations_test

2024-11-06 12:01:02.536330: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 12:01:02.582730: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 12:01:02.596304: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 12:01:02.634130: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 12:01:04.325129: W tensorflow/compiler/tf2

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/cleaned-data')

In [ ]:
translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/target-tok/trained_model/translations_gtest'

In [ ]:
# post-processing predictions
with open(translations_path, 'r', encoding='utf-8') as f:
  segments = f.read().splitlines()

for i in range(len(segments)):
  segments[i] = remove_whitespace_before(segments[i], before_punc)

post_translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/target-tok/trained_model/post_process_translations_gtest'
with open(post_translations_path, 'w') as f:
  for segment in segments:
    f.write(segment + '\n')

In [ ]:
!sacrebleu ref1.tso ref2.tso ref3.tso ref4.tso -i $translations_path -m bleu --force
!sacrebleu ref1.tso ref2.tso ref3.tso ref4.tso -i $post_translations_path -m bleu

{
 "name": "BLEU",
 "score": 28.8,
 "signature": "nrefs:4|case:mixed|eff:no|tok:13a|smooth:exp|version:2.4.3",
 "verbose_score": "66.3/38.9/23.1/13.4 (BP = 0.964 ratio = 0.964 hyp_len = 10915 ref_len = 11318)",
 "nrefs": "4",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.4.3"
}
{
 "name": "BLEU",
 "score": 28.8,
 "signature": "nrefs:4|case:mixed|eff:no|tok:13a|smooth:exp|version:2.4.3",
 "verbose_score": "66.3/38.9/23.1/13.4 (BP = 0.964 ratio = 0.964 hyp_len = 10915 ref_len = 11318)",
 "nrefs": "4",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.4.3"
}


In [ ]:
translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/target-tok/trained_model/translations_test'

In [ ]:
# post-processing predictions
with open(translations_path, 'r', encoding='utf-8') as f:
  segments = f.read().splitlines()

for i in range(len(segments)):
  segments[i] = remove_whitespace_before(segments[i], before_punc)

post_translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/target-tok/trained_model/post_process_translations_test'
with open(post_translations_path, 'w') as f:
  for segment in segments:
    f.write(segment + '\n')

In [ ]:
!sacrebleu test.tso -i $translations_path -m bleu --force
!sacrebleu test.tso -i $post_translations_path -m bleu

{
 "name": "BLEU",
 "score": 35.0,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.4.3",
 "verbose_score": "64.3/43.9/32.1/24.2 (BP = 0.910 ratio = 0.914 hyp_len = 28625 ref_len = 31326)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.4.3"
}
{
 "name": "BLEU",
 "score": 35.0,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.4.3",
 "verbose_score": "64.3/43.9/32.1/24.2 (BP = 0.910 ratio = 0.914 hyp_len = 28625 ref_len = 31326)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.4.3"
}
